## Machine Learning

This notebook summarizes the performance of various different machine learning models when applied to the dataset.

The first step I will take is to import some basic python packages and load the dataset into the `df` variable.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = 500

%matplotlib inline

In [2]:
df = pd.read_csv('df_model_5.csv', index_col=0)

In [5]:
df.head()

,casename,date,id,judge,opinion,type,judge_count,year,word_count,sentence_count,avg_sent_length,polarity,subjectivity
0,ACT-UP Triangle v. Commission for Health Services,1997-04-11,53839,frye,"FRYE, Justice.\nThis case involves the adoptio...",majority,415,1997,4364,214,20.392523,0.102371,0.473721
1,Mahoney v. Ronnie’s Road Service,1997-03-07,53841,per_curiam,PER CURIAM.\nAFFIRMED.\nJustice PARKER did not...,majority,6440,1997,16,3,5.333333,0.000000,0.000000
2,State v. Westbrooks,1996-12-06,53843,parker_sarah,"PARKER, Justice.\nDefendant, Donna Sue Westbro...",majority,456,1996,9467,405,23.375309,-0.012108,0.321622
3,State v. Conner,1997-02-10,53847,whichard,"WHICHARD, Justice.\nOn 13 November 1990, defen...",majority,661,1997,6111,268,22.802239,-0.024927,0.415121
4,Fulton Corp. v. Faulkner,1997-02-10,53848,webb,"WEBB, Justice.\nThis case brings to the Court ...",majority,801,1997,1346,75,17.946667,0.051909,0.415155


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80749 entries, 0 to 80748
Data columns (total 13 columns):
casename           80749 non-null object
date               80748 non-null object
id                 80749 non-null int64
judge              80749 non-null object
opinion            80749 non-null object
type               80749 non-null object
judge_count        80749 non-null int64
year               80749 non-null int64
word_count         80749 non-null int64
sentence_count     80749 non-null int64
avg_sent_length    80749 non-null float64
polarity           80749 non-null float64
subjectivity       80749 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 8.6+ MB


## Preparing the Features for Modeling

I will start by testing how some classifiers perform using only the characteristic features of the opinions that I used when exploring the data without the actual text of the opinions.  In order to do this, I need to convert those features into an appropriate format for the machine learning models.

### Creating the Machine Learning Dataframe

I will begin by creating a separate dataframe, `df_ml`, where I will store the various features that we will use in training the models.  I will start by adding the id number for each opinion for identification purposes, as well as the judge, which will be our label that the model will attempt to predict.

In [6]:
df_ml = df.loc[:, ['id', 'judge']]
df_ml.head()

,id,judge
0,53839,frye
1,53841,per_curiam
2,53843,parker_sarah
3,53847,whichard
4,53848,webb


### Converting the Categorical Features into Dummy Variables

The dataset has two categorical features, the type of opinion (majority, concurrence, etc.) and the year the opinion was issued.  I will convert both of these features into dummy columns using `pd.get_dummies` and then add the dummies to the `df_ml` dataframe:

In [7]:
# create dummy columns for 'type' feature
df_dummies = pd.get_dummies(df['type'], prefix='type')
df_dummies.head()

,type_concurrence,type_concurring-in-part-and-dissenting-in-part,type_dissent,type_majority,type_rehearing
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0


In [9]:
# add the dummy columns to the df_ml dataframe
df_ml = pd.concat([df_ml, df_dummies], axis=1)
df_ml.head()

,id,judge,type_concurrence,type_concurring-in-part-and-dissenting-in-part,type_dissent,type_majority,type_rehearing
0,53839,frye,0,0,0,1,0
1,53841,per_curiam,0,0,0,1,0
2,53843,parker_sarah,0,0,0,1,0
3,53847,whichard,0,0,0,1,0
4,53848,webb,0,0,0,1,0


In [10]:
# create dummy columns for 'year' feature
df_dummies = pd.get_dummies(df['year'], prefix='year')
df_dummies.head()

,year_1779,year_1784,year_1787,year_1789,year_1790,year_1791,year_1792,year_1793,year_1794,year_1795,year_1796,year_1797,year_1798,year_1799,year_1800,year_1801,year_1802,year_1803,year_1804,year_1805,year_1806,year_1807,year_1808,year_1809,year_1810,year_1811,year_1812,year_1813,year_1814,year_1815,year_1816,year_1817,year_1818,year_1819,year_1820,year_1821,year_1822,year_1823,year_1824,year_1825,year_1826,year_1827,year_1828,year_1829,year_1830,year_1831,year_1832,year_1833,year_1834,year_1835,year_1836,year_1837,year_1838,year_1839,year_1840,year_1841,year_1842,year_1843,year_1844,year_1845,year_1846,year_1847,year_1848,year_1849,year_1850,year_1851,year_1852,year_1853,year_1854,year_1855,year_1856,year_1857,year_1858,year_1859,year_1860,year_1861,year_1862,year_1863,year_1864,year_1866,year_1867,year_1868,year_1869,year_1870,year_1871,year_1872,year_1873,year_1874,year_1875,year_1876,year_1877,year_1878,year_1879,year_1880,year_1881,year_1882,year_1883,year_1884,year_1885,year_1886,year_1887,year_1888,year_1889,year_1890,year_1891,year_1892,year_1893,year_1894,year_1895,year_1896,year_1897,year_1898,year_1899,year_1900,year_1901,year_1902,year_1903,year_1904,year_1905,year_1906,year_1907,year_1908,year_1909,year_1910,year_1911,year_1912,year_1913,year_1914,year_1915,year_1916,year_1917,year_1918,year_1919,year_1920,year_1921,year_1922,year_1923,year_1924,year_1925,year_1926,year_1927,year_1928,year_1929,year_1930,year_1931,year_1932,year_1933,year_1934,year_1935,year_1936,year_1937,year_1938,year_1939,year_1940,year_1941,year_1942,year_1943,year_1944,year_1945,year_1946,year_1947,year_1948,year_1949,year_1950,year_1951,year_1952,year_1953,year_1954,year_1955,year_1956,year_1957,year_1958,year_1959,year_1960,year_1961,year_1962,year_1963,year_1964,year_1965,year_1966,year_1967,year_1968,year_1969,year_1970,year_1971,year_1972,year_1973,year_1974,year_1975,year_1976,year_1977,year_1978,year_1979,year_1980,year_1981,year_1982,year_1983,year_1984,year_1985,year_1986,year_1987,year_1988,year_1989,year_1990,year_1991,year_1992,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [11]:
# add the dummy columns to the df_ml dataframe
df_ml = pd.concat([df_ml, df_dummies], axis=1)
df_ml.head()

,id,judge,type_concurrence,type_concurring-in-part-and-dissenting-in-part,type_dissent,type_majority,type_rehearing,year_1779,year_1784,year_1787,year_1789,year_1790,year_1791,year_1792,year_1793,year_1794,year_1795,year_1796,year_1797,year_1798,year_1799,year_1800,year_1801,year_1802,year_1803,year_1804,year_1805,year_1806,year_1807,year_1808,year_1809,year_1810,year_1811,year_1812,year_1813,year_1814,year_1815,year_1816,year_1817,year_1818,year_1819,year_1820,year_1821,year_1822,year_1823,year_1824,year_1825,year_1826,year_1827,year_1828,year_1829,year_1830,year_1831,year_1832,year_1833,year_1834,year_1835,year_1836,year_1837,year_1838,year_1839,year_1840,year_1841,year_1842,year_1843,year_1844,year_1845,year_1846,year_1847,year_1848,year_1849,year_1850,year_1851,year_1852,year_1853,year_1854,year_1855,year_1856,year_1857,year_1858,year_1859,year_1860,year_1861,year_1862,year_1863,year_1864,year_1866,year_1867,year_1868,year_1869,year_1870,year_1871,year_1872,year_1873,year_1874,year_1875,year_1876,year_1877,year_1878,year_1879,year_1880,year_1881,year_1882,year_1883,year_1884,year_1885,year_1886,year_1887,year_1888,year_1889,year_1890,year_1891,year_1892,year_1893,year_1894,year_1895,year_1896,year_1897,year_1898,year_1899,year_1900,year_1901,year_1902,year_1903,year_1904,year_1905,year_1906,year_1907,year_1908,year_1909,year_1910,year_1911,year_1912,year_1913,year_1914,year_1915,year_1916,year_1917,year_1918,year_1919,year_1920,year_1921,year_1922,year_1923,year_1924,year_1925,year_1926,year_1927,year_1928,year_1929,year_1930,year_1931,year_1932,year_1933,year_1934,year_1935,year_1936,year_1937,year_1938,year_1939,year_1940,year_1941,year_1942,year_1943,year_1944,year_1945,year_1946,year_1947,year_1948,year_1949,year_1950,year_1951,year_1952,year_1953,year_1954,year_1955,year_1956,year_1957,year_1958,year_1959,year_1960,year_1961,year_1962,year_1963,year_1964,year_1965,year_1966,year_1967,year_1968,year_1969,year_1970,year_1971,year_1972,year_1973,year_1974,year_1975,year_1976,year_1977,year_1978,year_1979,year_1980,year_1981,year_1982,year_1983,year_1984,year_1985,year_1986,year_1987,year_1988,year_1989,year_1990,year_1991,year_1992,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017
0,53839,frye,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,53841,per_curiam,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,53843,parker_sarah,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,53847,whichard,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Normalize the Continuous Variables

The dataset has a number of continuous variables that I need to normalize, including the word count, sentence count, and average sentence lenghth.  Also, the polarity scores from `TextBlob` are on a -1-to-1 scale, so I will convert those values into a normalized 0-to-1 range.

In [15]:
# Import and instantiate MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

# Normalize the relevant features
x = df.iloc[:, 8:12].values
x_scaled = min_max_scaler.fit_transform(x)

# Save the normalized values into a dataframe
df_scaled = pd.DataFrame(x_scaled, columns=['word_count', 'sentence_count', 'avg_sent_length', 'polarity'])
df_scaled.head()

,word_count,sentence_count,avg_sent_length,polarity
0,0.088804,0.098247,0.093527,0.501317
1,0.000265,0.000923,0.018977,0.444444
2,0.192718,0.186347,0.108294,0.437718
3,0.124379,0.123155,0.105457,0.430596
4,0.027348,0.034133,0.081419,0.473283


In [17]:
# add the normalized features to the df_ml dataframe
df_ml = pd.concat([df_ml, df_scaled], axis=1)

# add the already-normalized subjectivity column to the dl_ml dataframe
df_ml['subjectivity'] = df.loc[:, 'subjectivity']

df_ml.head()

,id,judge,type_concurrence,type_concurring-in-part-and-dissenting-in-part,type_dissent,type_majority,type_rehearing,year_1779,year_1784,year_1787,year_1789,year_1790,year_1791,year_1792,year_1793,year_1794,year_1795,year_1796,year_1797,year_1798,year_1799,year_1800,year_1801,year_1802,year_1803,year_1804,year_1805,year_1806,year_1807,year_1808,year_1809,year_1810,year_1811,year_1812,year_1813,year_1814,year_1815,year_1816,year_1817,year_1818,year_1819,year_1820,year_1821,year_1822,year_1823,year_1824,year_1825,year_1826,year_1827,year_1828,year_1829,year_1830,year_1831,year_1832,year_1833,year_1834,year_1835,year_1836,year_1837,year_1838,year_1839,year_1840,year_1841,year_1842,year_1843,year_1844,year_1845,year_1846,year_1847,year_1848,year_1849,year_1850,year_1851,year_1852,year_1853,year_1854,year_1855,year_1856,year_1857,year_1858,year_1859,year_1860,year_1861,year_1862,year_1863,year_1864,year_1866,year_1867,year_1868,year_1869,year_1870,year_1871,year_1872,year_1873,year_1874,year_1875,year_1876,year_1877,year_1878,year_1879,year_1880,year_1881,year_1882,year_1883,year_1884,year_1885,year_1886,year_1887,year_1888,year_1889,year_1890,year_1891,year_1892,year_1893,year_1894,year_1895,year_1896,year_1897,year_1898,year_1899,year_1900,year_1901,year_1902,year_1903,year_1904,year_1905,year_1906,year_1907,year_1908,year_1909,year_1910,year_1911,year_1912,year_1913,year_1914,year_1915,year_1916,year_1917,year_1918,year_1919,year_1920,year_1921,year_1922,year_1923,year_1924,year_1925,year_1926,year_1927,year_1928,year_1929,year_1930,year_1931,year_1932,year_1933,year_1934,year_1935,year_1936,year_1937,year_1938,year_1939,year_1940,year_1941,year_1942,year_1943,year_1944,year_1945,year_1946,year_1947,year_1948,year_1949,year_1950,year_1951,year_1952,year_1953,year_1954,year_1955,year_1956,year_1957,year_1958,year_1959,year_1960,year_1961,year_1962,year_1963,year_1964,year_1965,year_1966,year_1967,year_1968,year_1969,year_1970,year_1971,year_1972,year_1973,year_1974,year_1975,year_1976,year_1977,year_1978,year_1979,year_1980,year_1981,year_1982,year_1983,year_1984,year_1985,year_1986,year_1987,year_1988,year_1989,year_1990,year_1991,year_1992,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,word_count,sentence_count,avg_sent_length,polarity,word_count,sentence_count,avg_sent_length,polarity,subjectivity
0,53839,frye,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.088804,0.098247,0.093527,0.501317,0.088804,0.098247,0.093527,0.501317,0.473721
1,53841,per_curiam,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000265,0.000923,0.018977,0.444444,0.000265,0.000923,0.018977,0.444444,0.000000
2,53843,parker_sarah,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [18]:
# save the df_ml dataframe to the local drive
export_df = df_ml.to_csv('df_ml_1.csv')

## Training Some Benchmark Models

In [36]:
df_ml2 = df_ml[df_ml['judge'] != 'per_curiam']
df_ml2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74309 entries, 0 to 80747
Columns: 247 entries, id to subjectivity
dtypes: float64(9), int64(1), object(1), uint8(236)
memory usage: 23.5+ MB


In [45]:
# divide the data into label and features for use in ml models
X = df_ml.iloc[:, 2:]
y = df_ml.loc[:, 'judge']
X.columns

Index(['type_concurrence', 'type_concurring-in-part-and-dissenting-in-part',
       'type_dissent', 'type_majority', 'type_rehearing', 'year_1779',
       'year_1784', 'year_1787', 'year_1789', 'year_1790',
       ...
       'year_2017', 'word_count', 'sentence_count', 'avg_sent_length',
       'polarity', 'word_count', 'sentence_count', 'avg_sent_length',
       'polarity', 'subjectivity'],
      dtype='object', length=245)

In [47]:
# split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, stratify=y)

### Logistic Regression

In [48]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=6)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print('Accuracy on training set = {}'.format(lr.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(lr.score(X_test, y_test)))

C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy on training set = 0.28983999603705357
Accuracy on test set = 0.2698632851198732


### Stochastic Gradient Descent Classifier

In [49]:
from sklearn.linear_model import SGDClassifier
sgdc = SGDClassifier(random_state=6)
sgdc.fit(X_train, y_train)
y_pred_sgdc = sgdc.predict(X_test)
print('Accuracy on training set = {}'.format(sgdc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(sgdc.score(X_test, y_test)))

Accuracy on training set = 0.23133699905880023
Accuracy on test set = 0.22359817713493163


### Linear Support Vector Machine Classifier

In [50]:
from sklearn.svm import LinearSVC
svc = LinearSVC(random_state=6)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print('Accuracy on training set = {}'.format(svc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(svc.score(X_test, y_test)))

Accuracy on training set = 0.302174666864814
Accuracy on test set = 0.2855656825837131


### Multinomial Naive Bayes

In [51]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print('Accuracy on training set = {}'.format(nb.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(nb.score(X_test, y_test)))

Accuracy on training set = 0.2104324565314311
Accuracy on test set = 0.20001981375074301


### Random Forest Classifier

In [52]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=6)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('Accuracy on training set = {}'.format(rf.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rf.score(X_test, y_test)))

C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy on training set = 0.9885404798467661
Accuracy on test set = 0.24148008718050326


### Summary

Here is a summary of the results:

| Model               | Train  | Test   |
|---------------------|--------|--------|
| Linear SVM          | 0.3022 | 0.2856 |
| Logistic Regression | 0.2898 | 0.2699 |
| Random Forest       | 0.9885 | 0.2415 |
| SGD                 | 0.2313 | 0.2234 |
| Naive Bayes         | 0.2104 | 0.2000 |

The best performing models were able to correctly predict the judge who wrote the opinion just over 25% of the time on the test set.  The top three performing models also appear to have overfit on the training set (the Random Forest Classifier to a much more significant degree).  While this is far from our ultimate goal, it does demonstrate that these features contain some signal.

Next, I will use `TfidfVectorizer` to vectorize the text of the opinions into additional features to see if it improves model performance.

## Vectorized Text Features

As discussed in the Data Preprocessing notebook, I have already used `TfidfVectorizer` on the corpus with varying `min_df` values to build vocabularies.  I will read in the vectorized text files and see how the models perform.  I will first test the smallest vocabulary of 3,677 words/features. 

In [3]:
# read in the saved tfidf vectors from local .csv file
df_tfidf = pd.read_csv('df_tfidf_02.csv', index_col=0)

In [5]:
df_tfidf.head()

,00,000,10,100,101,102,103,104,105,106,107,108,109,11,110,111,112,113,114,115,116,117,118,119,12,120,121,122,123,124,125,126,127,128,129,13,130,131,132,133,134,135,136,137,138,139,14,140,141,142,143,144,145,146,147,148,149,15,150,151,152,153,154,155,156,157,158,159,15a,16,160,161,162,163,164,165,166,167,168,169,17,170,171,172,173,174,175,176,177,178,179,18,180,181,182,183,184,185,186,187,188,189,19,190,191,192,193,194,195,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,196,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,197,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,198,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,199,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,1a,1st,20,200,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,201,2010,202,203,204,205,206,207,208,209,21,210,211,212,213,214,215,216,217,218,219,22,220,221,222,223,224,225,226,227,228,229,23,230,231,232,233,234,235,236,237,238,239,24,240,241,242,243,244,245,246,247,248,249,25,250,251,252,253,254,255,256,257,258,259,26,260,261,262,263,264,265,266,...,threat,threatened,three,through,throughout,thus,time,timely,times,tion,tire,title,to,today,together,told,too,took,top,tort,total,totally,toward,towards,town,track,tract,trade,traffic,train,training,transaction,transactions,transcript,transfer,transferred,transportation,travel,traveling,treat,treated,treating,treatment,trespass,trial,trials,tried,trouble,truck,true,trust,trustee,trustees,truth,try,trying,turn,turned,turner,twelve,twenty,twice,two,type,types,ultimate,ultimately,unable,uncertain,unconstitutional,uncontradicted,under,underlying,understand,understanding,understood,undertake,undertaking,undertook,undisputed,undoubtedly,undue,unfair,uniform,union,unit,united,university,unjust,unknown,unlawful,unlawfully,unless,unlike,unnecessary,unreasonable,unsupported,until,unusual,up,upheld,upon,urged,us,use,used,uses,using,usual,usually,va,vacate,vacated,valid,validity,valuable,value,variance,various,vehicle,vehicles,venire,verdict,verdicts,verified,version,very,vested,vi,victim,view,viewed,views,violate,violated,violates,violating,violation,violations,violence,virginia,virtue,visit,void,voir,vol,voluntarily,voluntary,vote,waive,waived,waiver,wake,walked,walking,wall,want,wanted,ward,warning,warrant,warranted,warrants,warranty,was,washington,water,watson,way,ways,wbicb,we,weapon,week,weeks,weigh,weight,welfare,well,went,were,west,what,whatever,whatsoever,when,whenever,where,whereas,whereby,wherein,whether,which,while,white,who,whole,wholly,whom,whose,why,wide,widow,wife,will,willful,willfully,william,willing,wilmington,wilson,window,winston,wish,wit,with,withdraw,within,without,withstand,witness,witnesses,woman,wood,word,words,work,worked,worker,workers,working,works,worth,would,wright,writ,writing,written,wrong,wrongful,wrongfully,wrote,year,years,yes,yet,york,you,young,your
0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.006115,0.0,0.017759,0.000000,0.0,0.000000,0.017337,0.00000,0.008272,0.0,0.0,0.011754,0.006137,0.01231,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.006142,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.018073,0.005839,0.0,0.000000,0.000000,0.0,0.0,0.004249,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.052304,0.004528,0.000000,0.005937,0.0,0.000000,0.0,0.00617,0.0,0.0,0.000000,0.0,0.004638,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.009079,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.009345,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.016766,0.000000,0.000000,0.000000,0.000000,0.005310,0.000000,0.005212,0.005344,0.000000,0.005347,0.005651,0.000000,0.005824,0.006144,0.000000,0.005732,0.005868,0.005840,0.116410,0.059592,0.030187,0.0,0.0,0.0,0.0,0.0,0.029396,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [4]:
# create the machine learing dataframe
df_ml = df.loc[:, ['id', 'judge']]
df_ml.head()

,id,judge
0,53839,frye
1,53841,per_curiam
2,53843,parker_sarah
3,53847,whichard
4,53848,webb


In [7]:
# add the vectorized vocabulary to the df_ml dataframe
df_ml = pd.concat([df_ml, df_tfidf], axis=1)

In [8]:
df_ml.head()

,id,judge,00,000,10,100,101,102,103,104,105,106,107,108,109,11,110,111,112,113,114,115,116,117,118,119,12,120,121,122,123,124,125,126,127,128,129,13,130,131,132,133,134,135,136,137,138,139,14,140,141,142,143,144,145,146,147,148,149,15,150,151,152,153,154,155,156,157,158,159,15a,16,160,161,162,163,164,165,166,167,168,169,17,170,171,172,173,174,175,176,177,178,179,18,180,181,182,183,184,185,186,187,188,189,19,190,191,192,193,194,195,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,196,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,197,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,198,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,199,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,1a,1st,20,200,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,201,2010,202,203,204,205,206,207,208,209,21,210,211,212,213,214,215,216,217,218,219,22,220,221,222,223,224,225,226,227,228,229,23,230,231,232,233,234,235,236,237,238,239,24,240,241,242,243,244,245,246,247,248,249,25,250,251,252,253,254,255,256,257,258,259,26,260,261,262,263,264,...,threat,threatened,three,through,throughout,thus,time,timely,times,tion,tire,title,to,today,together,told,too,took,top,tort,total,totally,toward,towards,town,track,tract,trade,traffic,train,training,transaction,transactions,transcript,transfer,transferred,transportation,travel,traveling,treat,treated,treating,treatment,trespass,trial,trials,tried,trouble,truck,true,trust,trustee,trustees,truth,try,trying,turn,turned,turner,twelve,twenty,twice,two,type,types,ultimate,ultimately,unable,uncertain,unconstitutional,uncontradicted,under,underlying,understand,understanding,understood,undertake,undertaking,undertook,undisputed,undoubtedly,undue,unfair,uniform,union,unit,united,university,unjust,unknown,unlawful,unlawfully,unless,unlike,unnecessary,unreasonable,unsupported,until,unusual,up,upheld,upon,urged,us,use,used,uses,using,usual,usually,va,vacate,vacated,valid,validity,valuable,value,variance,various,vehicle,vehicles,venire,verdict,verdicts,verified,version,very,vested,vi,victim,view,viewed,views,violate,violated,violates,violating,violation,violations,violence,virginia,virtue,visit,void,voir,vol,voluntarily,voluntary,vote,waive,waived,waiver,wake,walked,walking,wall,want,wanted,ward,warning,warrant,warranted,warrants,warranty,was,washington,water,watson,way,ways,wbicb,we,weapon,week,weeks,weigh,weight,welfare,well,went,were,west,what,whatever,whatsoever,when,whenever,where,whereas,whereby,wherein,whether,which,while,white,who,whole,wholly,whom,whose,why,wide,widow,wife,will,willful,willfully,william,willing,wilmington,wilson,window,winston,wish,wit,with,withdraw,within,without,withstand,witness,witnesses,woman,wood,word,words,work,worked,worker,workers,working,works,worth,would,wright,writ,writing,written,wrong,wrongful,wrongfully,wrote,year,years,yes,yet,york,you,young,your
0,53839,frye,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.006115,0.0,0.017759,0.000000,0.0,0.000000,0.017337,0.00000,0.008272,0.0,0.0,0.011754,0.006137,0.01231,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.006142,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.018073,0.005839,0.0,0.000000,0.000000,0.0,0.0,0.004249,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.052304,0.004528,0.000000,0.005937,0.0,0.000000,0.0,0.00617,0.0,0.0,0.000000,0.0,0.004638,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.009079,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.009345,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.016766,0.000000,0.000000,0.000000,0.000000,0.005310,0.000000,0.005212,0.005344,0.000000,0.005347,0.005651,0.000000,0.005824,0.006144,0.000000,0.005732,0.005868,0.005840,0.116410,0.059592,0.030187,0.0,0.0,0.0,0.0,0.0,0.029396,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.

In [9]:
# divide the data into label and features for use in ml models
X = df_ml.iloc[:, 2:]
y = df_ml.loc[:, 'judge']
X.columns

Index(['00', '000', '10', '100', '101', '102', '103', '104', '105', '106',
       ...
       'wrongfully', 'wrote', 'year', 'years', 'yes', 'yet', 'york', 'you',
       'young', 'your'],
      dtype='object', length=3677)

In [11]:
# split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, stratify=y)

### Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=6)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print('Accuracy on training set = {}'.format(lr.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(lr.score(X_test, y_test)))

C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy on training set = 0.581562391638183
Accuracy on test set = 0.49217356845650884


### Stochastic Gradient Descent Classifier

In [13]:
from sklearn.linear_model import SGDClassifier
sgdc = SGDClassifier(random_state=6)
sgdc.fit(X_train, y_train)
y_pred_sgdc = sgdc.predict(X_test)
print('Accuracy on training set = {}'.format(sgdc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(sgdc.score(X_test, y_test)))

Accuracy on training set = 0.7290335364343389
Accuracy on test set = 0.5581038240538934


### Linear Support Vector Machine Classifier

In [14]:
from sklearn.svm import LinearSVC
svc = LinearSVC(random_state=6)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print('Accuracy on training set = {}'.format(svc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(svc.score(X_test, y_test)))

Accuracy on training set = 0.9444361883060055
Accuracy on test set = 0.6814444224291658


### Multinomial Naive Bayes

In [16]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print('Accuracy on training set = {}'.format(nb.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(nb.score(X_test, y_test)))

Accuracy on training set = 0.23239378477898318
Accuracy on test set = 0.21111551416683177


### Random Forest Classifier

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=900, max_depth=9, min_samples_split=600, min_samples_leaf=50, random_state=6)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('Accuracy on training set = {}'.format(rf.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rf.score(X_test, y_test)))

Accuracy on training set = 0.24644573240204093
Accuracy on test set = 0.22770953041410738


### Summary

Here is a summary of the results:

| Model (min_df=0.02) | Train  | Test   |
|---------------------|--------|--------|
| Linear SVM          | 0.9044 | 0.6814 |
| Logistic Regression | 0.5816 | 0.4922 |
| SGD                 | 0.7290 | 0.5581 |
| Random Forest       | 0.2464 | 0.2277 |
| Naive Bayes         | 0.2324 | 0.2111 |

Three of the classifiers performed substantially better with the vectorized text.  I will drop the Random Forest and Naive Bayes models given their inferior performance.  Let's see how the Linear SVM, Logistic Regression, and SGD models do with an expanded vocabulary of 16905 words/features:

In [3]:
# create stop words for use in vectorizer
stop_words = list(set(df['judge']))
add_stop_words = ['clark', 'johnson', 'martin', 'parker', 'timmons', 'goodson', 'walker', 'judge', 'justice', 'per', 'curiam']
stop_words += add_stop_words

In [4]:
# vectorize the corpus using TfidfVectorizer and a min_df of 0.001
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop_words, min_df=0.001)
data_tfidf = tfidf.fit_transform(df['opinion'])
df_tfidf = pd.DataFrame(data_tfidf.toarray(), columns=tfidf.get_feature_names())
df_tfidf.index = df.index

df_tfidf.head()

,00,000,01,02,03,04,05,050,06,07,08,09,0f,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,1035,1036,1037,1038,1039,104,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,105,1050,1051,1052,1053,1054,1055,1056,1057,1059,106,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,107,1070,1072,1073,1074,1075,1076,1077,1078,1079,108,1080,1081,1082,1083,1084,1086,1087,1089,108a,109,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,10a,10th,11,110,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,111,1110,1111,1112,1113,1114,1117,112,1120,1122,1123,1124,1125,1126,1127,1128,1129,113,1130,1131,1133,1134,1135,1137,1138,1139,113a,114,1143,1145,1147,1148,1149,115,1152,1154,1156,1159,115c,116,1160,1161,1164,1165,1167,1169,117,1178,118,1180,1181,1183,1189,119,1194,1197,1199,11th,12,120,1200,1201,1205,1206,1207,1208,1209,121,1210,1211,1212,1213,1214,1215,122,1221,1222,1225,1226,1227,1228,1229,122c,123,1230,1231,1232,1233,1234,1235,1236,1237,124,1240,1241,1242,1243,1245,1246,1247,1249,125,1250,1251,1253,1254,126,127,128,1283,129,12th,13,130,1300,1302,...,willingness,willis,williston,willoughby,wills,wilmington,wilson,win,winbobne,winboene,winchester,wind,winders,windfall,winding,windley,window,windows,winds,windshield,wine,winfield,wing,wingo,winn,winner,winning,winslow,winstead,winston,winter,winters,wipe,wiped,wire,wired,wires,wiring,wis,wisconsin,wisdom,wise,wisely,wiseman,wiser,wisest,wish,wished,wishes,wishing,wit,witb,witbin,with,withdraw,withdrawal,withdrawals,withdrawing,withdrawn,withdraws,withdrew,withers,witherspoon,withheld,withhold,withholding,withholds,within,without,withstand,witness,witnessed,witnesses,witnessing,witt,wives,wjhere,wjhether,wl,wm,wms,wo,woke,wolf,wolfe,womack,woman,womble,women,won,wonder,wong,wood,woodard,wooded,wooden,woodfin,woodhouse,woodland,woodlief,woodmen,woodruff,woods,woodson,woodward,woody,wool,woolard,wooten,word,worded,wording,words,wore,work,workable,worked,worker,workers,working,workman,workmanlike,workmanship,workmen,workplace,works,worksheet,worland,world,worley,worn,worried,worry,worse,worsened,worship,worsley,worst,worth,wortham,worthington,worthless,worthy,would,wouldn,wound,wounded,wounding,wounds,wrapped,wras,wray,wreck,wrecked,wrecker,wrenn,wright,wrightsville,wrist,wrists,writ,write,writer,writers,writes,writing,writings,writs,written,wrong,wrongdoer,wrongdoers,wrongdoing,wronged,wrongful,wrongfully,wrongly,wrongs,wrote,wrought,wyatt,wynne,wyo,wás,wé,xi,xii,xiii,xiv,ya,yadkin,yale,yance,yancey,yarborough,yarbrough,yard,yards,yarn,yates,ye,yeah,year,yearly,years,yell,yelled,yelling,yellow,yelverton,yes,yesterday,yet,yield,yielded,yielding,yields,yii,yirginia,yol,yon,york,you,young,youngblood,younger,youngest,yount,your,yours,yourself,yourselves,youth,youthful,yow,zachary,zeal,zealous,zero,zimmerman,zone,zoned,zones,zoning,zuniga,zurich,ánd,áre,ás,óf
0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015969,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.015589,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.007438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005518,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [5]:
# create the machine learing dataframe
df_ml = df.loc[:, ['id', 'judge']]

# add the vectorized vocabulary to the df_ml dataframe
df_ml = pd.concat([df_ml, df_tfidf], axis=1)

In [6]:
# divide the data into label and features for use in ml models
X = df_ml.iloc[:, 2:]
y = df_ml.loc[:, 'judge']

# split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, stratify=y)

### Logistic Regression

In [7]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=6)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print('Accuracy on training set = {}'.format(lr.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(lr.score(X_test, y_test)))

C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy on training set = 0.6063638315087267
Accuracy on test set = 0.4777095304141074


### Stochastic Gradient Descent Classifier

In [8]:
from sklearn.linear_model import SGDClassifier
sgdc = SGDClassifier(random_state=6)
sgdc.fit(X_train, y_train)
y_pred_sgdc = sgdc.predict(X_test)
print('Accuracy on training set = {}'.format(sgdc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(sgdc.score(X_test, y_test)))

Accuracy on training set = 0.8424398540314724
Accuracy on test set = 0.5780661779274817


### Linear Support Vector Machine Classifier

In [9]:
from sklearn.svm import LinearSVC
svc = LinearSVC(random_state=6)
svc.fit(X_train, y_train)

y_pred_svc = svc.predict(X_test)
print('Accuracy on training set = {}'.format(svc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(svc.score(X_test, y_test)))

Accuracy on training set = 0.9884909430161325
Accuracy on test set = 0.6752526253219735


### Summary

Here is a summary of the results:

| Model               | Train (min_df=0.02) | Test (min_df=0.02) | Train (min_df=0.001) | Test (min_df=0.001) |
|---------------------|---------------------|--------------------|----------------------|---------------------|
| Linear SVM          | 0.9044              | 0.6814             | 0.9885               | 0.6753              |
| Logistic Regression | 0.5816              | 0.4922             | 0.6064               | 0.4778              |
| SGD                 | 0.7290              | 0.5581             | 0.8424               | 0.5781              |

Expanding the vocabulary from 3677 to 16905 words did not meaningfully improve model performance.  The Linear SVM and Logistic Regression models actually performed worse on the test set and the SGD Classifier improved by only 2%.  This isn't necessarily surprising and demonstrates that expanding the vocabulary even further is not likely to improve model performance, as it is adding less-utilized, and likely less-significant, words to the vocabulary.  Expanding the vocabulary further would also open the door for more OCR errors to be included, which we know is an issue with this dataset.

For these reasons, I will proceed with the 3,677 word vocabulary, as it will make model building more efficient without sacrificing much, if any, performance.

### Combined Feature Performance

Next, I will combine the categorical and continuous variables with the tfidf-vectorized vocabulary to see how the models perform with both sets of features.

In [2]:
# load the categorical and continuous variables
df_ml = pd.read_csv('df_ml_1.csv', index_col=0)
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80749 entries, 0 to 80748
Columns: 243 entries, id to subjectivity
dtypes: float64(5), int64(237), object(1)
memory usage: 150.3+ MB


In [3]:
# load the tfidf vectorized vocabulary
df_tfidf = pd.read_csv('df_tfidf_02.csv', index_col=0)
df_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80749 entries, 0 to 80748
Columns: 3677 entries, 00 to your
dtypes: float64(3677)
memory usage: 2.2 GB


In [5]:
# add the vectorized vocabulary to the df_ml dataframe
df_ml = pd.concat([df_ml, df_tfidf], axis=1)
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80749 entries, 0 to 80748
Columns: 3920 entries, id to your
dtypes: float64(3682), int64(237), object(1)
memory usage: 2.4+ GB


In [6]:
df_ml.head()

,id,judge,type_concurrence,type_concurring-in-part-and-dissenting-in-part,type_dissent,type_majority,type_rehearing,year_1779,year_1784,year_1787,year_1789,year_1790,year_1791,year_1792,year_1793,year_1794,year_1795,year_1796,year_1797,year_1798,year_1799,year_1800,year_1801,year_1802,year_1803,year_1804,year_1805,year_1806,year_1807,year_1808,year_1809,year_1810,year_1811,year_1812,year_1813,year_1814,year_1815,year_1816,year_1817,year_1818,year_1819,year_1820,year_1821,year_1822,year_1823,year_1824,year_1825,year_1826,year_1827,year_1828,year_1829,year_1830,year_1831,year_1832,year_1833,year_1834,year_1835,year_1836,year_1837,year_1838,year_1839,year_1840,year_1841,year_1842,year_1843,year_1844,year_1845,year_1846,year_1847,year_1848,year_1849,year_1850,year_1851,year_1852,year_1853,year_1854,year_1855,year_1856,year_1857,year_1858,year_1859,year_1860,year_1861,year_1862,year_1863,year_1864,year_1866,year_1867,year_1868,year_1869,year_1870,year_1871,year_1872,year_1873,year_1874,year_1875,year_1876,year_1877,year_1878,year_1879,year_1880,year_1881,year_1882,year_1883,year_1884,year_1885,year_1886,year_1887,year_1888,year_1889,year_1890,year_1891,year_1892,year_1893,year_1894,year_1895,year_1896,year_1897,year_1898,year_1899,year_1900,year_1901,year_1902,year_1903,year_1904,year_1905,year_1906,year_1907,year_1908,year_1909,year_1910,year_1911,year_1912,year_1913,year_1914,year_1915,year_1916,year_1917,year_1918,year_1919,year_1920,year_1921,year_1922,year_1923,year_1924,year_1925,year_1926,year_1927,year_1928,year_1929,year_1930,year_1931,year_1932,year_1933,year_1934,year_1935,year_1936,year_1937,year_1938,year_1939,year_1940,year_1941,year_1942,year_1943,year_1944,year_1945,year_1946,year_1947,year_1948,year_1949,year_1950,year_1951,year_1952,year_1953,year_1954,year_1955,year_1956,year_1957,year_1958,year_1959,year_1960,year_1961,year_1962,year_1963,year_1964,year_1965,year_1966,year_1967,year_1968,year_1969,year_1970,year_1971,year_1972,year_1973,year_1974,year_1975,year_1976,year_1977,year_1978,year_1979,year_1980,year_1981,year_1982,year_1983,year_1984,year_1985,year_1986,year_1987,year_1988,year_1989,year_1990,year_1991,year_1992,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,word_count,sentence_count,avg_sent_length,polarity,subjectivity,00,000,10,100,101,102,103,...,threat,threatened,three,through,throughout,thus,time,timely,times,tion,tire,title,to,today,together,told,too,took,top,tort,total,totally,toward,towards,town,track,tract,trade,traffic,train,training,transaction,transactions,transcript,transfer,transferred,transportation,travel,traveling,treat,treated,treating,treatment,trespass,trial,trials,tried,trouble,truck,true,trust,trustee,trustees,truth,try,trying,turn,turned,turner,twelve,twenty,twice,two,type,types,ultimate,ultimately,unable,uncertain,unconstitutional,uncontradicted,under,underlying,understand,understanding,understood,undertake,undertaking,undertook,undisputed,undoubtedly,undue,unfair,uniform,union,unit,united,university,unjust,unknown,unlawful,unlawfully,unless,unlike,unnecessary,unreasonable,unsupported,until,unusual,up,upheld,upon,urged,us,use,used,uses,using,usual,usually,va,vacate,vacated,valid,validity,valuable,value,variance,various,vehicle,vehicles,venire,verdict,verdicts,verified,version,very,vested,vi,victim,view,viewed,views,violate,violated,violates,violating,violation,violations,violence,virginia,virtue,visit,void,voir,vol,voluntarily,voluntary,vote,waive,waived,waiver,wake,walked,walking,wall,want,wanted,ward,warning,warrant,warranted,warrants,warranty,was,washington,water,watson,way,ways,wbicb,we,weapon,week,weeks,weigh,weight,welfare,well,went,were,west,what,whatever,whatsoever,when,whenever,where,whereas,whereby,wherein,whether,which,while,white,who,whole,wholly,whom,whose,why,wide

In [7]:
# divide the data into label and features for use in ml models
X = df_ml.iloc[:, 2:]
y = df_ml.loc[:, 'judge']

# split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, stratify=y)

### Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=6)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print('Accuracy on training set = {}'.format(lr.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(lr.score(X_test, y_test)))

C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy on training set = 0.7363649873681082
Accuracy on test set = 0.6236378046364177


### Stochastic Gradient Descent Classifier

In [9]:
from sklearn.linear_model import SGDClassifier
sgdc = SGDClassifier(random_state=6)
sgdc.fit(X_train, y_train)
y_pred_sgdc = sgdc.predict(X_test)
print('Accuracy on training set = {}'.format(sgdc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(sgdc.score(X_test, y_test)))

Accuracy on training set = 0.7165007182840442
Accuracy on test set = 0.5903011690112938


### Linear Support Vector Machine Classifier

In [10]:
from sklearn.svm import LinearSVC
svc = LinearSVC(random_state=6)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print('Accuracy on training set = {}'.format(svc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(svc.score(X_test, y_test)))

Accuracy on training set = 0.9654563167715197
Accuracy on test set = 0.7418763621953636


### Summary

Here are the results:

| Model               | Train  | Test   |
|---------------------|--------|--------|
| Linear SVM          | 0.9655 | 0.7419 |
| Logistic Regression | 0.7364 | 0.6236 |
| SGD                 | 0.7165 | 0.5903 |

Combining the features improved the accuracy of the Linear SVM and Logistic Regression models significantly.  Both models are producing favorable accuracy percentages considering that the model has to identify the correct author from a pool of over 150 judges.  The SGD Classifier did not improve as significantly as the other two models and is now producing the lowest accuracy, so I have elected to move forward without it.

## Model Hyperparameter Tuning

Now that I have selected the two best-performing classifiers, I will do some model tuning to see if that further improves their performance.

### Logistic Regression

First, I will try the `newton-cg` solver to see if it will increase model performance over the default `lbfgs` solver:

In [ ]:
from sklearn.linear_model import LogisticRegression
solver_list = ['newton-cg']
for solver in solver_list:
    lr = LogisticRegression(solver=solver, multi_class='auto')
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    print('Solver value {} Training Set Accuracy: {}'.format(solver, lr.score(X_train, y_train)))
    print('Solver value {} Test Set Accuracy: {}'.format(solver, lr.score(X_test, y_test)))

Solver value newton-cg Training Set Accuracy: 0.7517214048645168
Solver value newton-cg Test Set Accuracy: 0.6322072518327719


The `newton-cg` solver produced the same accuracy results, so I will stick with the recommended default `lbfgs` solver.  Next, I will tune the C parameter, which controls the regularization strength of the model:

In [8]:
from sklearn.linear_model import LogisticRegression
c_list = [0.01, 0.1, 1, 10, 100, 1000, 10000]
for c in c_list:
    lr = LogisticRegression(C=c, solver='lbfgs', multi_class='auto', max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    print('C value {} Training Set Accuracy: {}'.format(c, lr.score(X_train, y_train)))
    print('C value {} Test Set Accuracy: {}'.format(c, lr.score(X_test, y_test)))

C value 0.01 Training Set Accuracy: 0.13835636795957795
C value 0.01 Test Set Accuracy: 0.13552605508222706
C value 0.1 Training Set Accuracy: 0.41714965076534405
C value 0.1 Test Set Accuracy: 0.37859124232217156
C value 1 Training Set Accuracy: 0.7516718680338832
C value 1 Test Set Accuracy: 0.6322072518327719
C value 10 Training Set Accuracy: 0.9649609484651839
C value 10 Test Set Accuracy: 0.7231523677432138


C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


C value 100 Training Set Accuracy: 0.9997027790161985
C value 100 Test Set Accuracy: 0.7294927679809788


C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


C value 1000 Training Set Accuracy: 0.9999009263387328
C value 1000 Test Set Accuracy: 0.7243907271646522


C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


C value 10000 Training Set Accuracy: 0.9999009263387328
C value 10000 Test Set Accuracy: 0.720229839508619


Here is a summary of the results of our hyperparameter tuning for the Logistic Regression classifier:

| Model               | Solver    | C Value | Train  | Test   |
|---------------------|-----------|---------|--------|--------|
| Logistic Regression | lbfgs     | 100     | 0.9998 | 0.7295 |
| Logistic Regression | lbfgs     | 1000    | 0.9999 | 0.7244 |
| Logistic Regression | lbfgs     | 10      | 0.9650 | 0.7232 |
| Logistic Regression | lbfgs     | 10000   | 0.9999 | 0.7202 |
| Logistic Regression | lbfgs     | 1       | 0.7517 | 0.6322 |
| Logistic Regression | newton-cg | 1       | 0.7517 | 0.6322 |
| Logistic Regression | lbfgs     | 0.1     | 0.4171 | 0.3786 |
| Logistic Regression | lbfgs     | 0.001   | 0.1384 | 0.1355 |

The tuning suggests that a C parameter value of 100 will result in the marginally best performance of the model.  I also will need to increase the number of iterations to higher than 1000 to allow the model to converge using that C parameter.

### Linear Support Vector Machine Classifier

For the Linear Support Vector Machine Classifier, I will tune the C parameter:

In [11]:
from sklearn.svm import LinearSVC
c_list = [0.01, 0.1, 1, 10, 100, 1000]
for c in c_list:
    svc = LinearSVC(C=c, max_iter=1500, random_state=6)
    svc.fit(X_train, y_train)
    y_pred_svc = svc.predict(X_test)
    print('C value {} Training Set Accuracy: {}'.format(c, svc.score(X_train, y_train)))
    print('C value {} Test Set Accuracy: {}'.format(c, svc.score(X_test, y_test)))

C value 0.01 Training Set Accuracy: 0.5204999917438615
C value 0.01 Test Set Accuracy: 0.456905092133941
C value 0.1 Training Set Accuracy: 0.8104885982728158
C value 0.1 Test Set Accuracy: 0.6641569249058847
C value 1 Training Set Accuracy: 0.9654563167715197
C value 1 Test Set Accuracy: 0.7418763621953636
C value 10 Training Set Accuracy: 0.9975561830220769
C value 10 Test Set Accuracy: 0.7231523677432138


C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


C value 100 Training Set Accuracy: 0.9995706808011757
C value 100 Test Set Accuracy: 0.6945214979195562


C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


C value 1000 Training Set Accuracy: 0.9995871930780535
C value 1000 Test Set Accuracy: 0.6842678819100456


It appears that the default C parameter value of 1 produced the highest accuracy for the Linear SVC model.

### Summary of Hyperparameter Tuning

Here is a summary of the results of the hyperparameter tuning:

| Model               | Solver    | C Value | Train  | Test   |
|---------------------|-----------|---------|--------|--------|
| Linear SVC          |           | 1       | 0.9655 | 0.7419 |
| Logistic Regression | lbfgs     | 100     | 0.9998 | 0.7295 |
| Logistic Regression | lbfgs     | 1000    | 0.9999 | 0.7244 |
| Linear SVC          |           | 10      | 0.9976 | 0.7232 |
| Logistic Regression | lbfgs     | 10      | 0.9650 | 0.7232 |
| Logistic Regression | lbfgs     | 10000   | 0.9999 | 0.7202 |
| Linear SVC          |           | 100     | 0.9996 | 0.6945 |
| Linear SVC          |           | 1000    | 0.9996 | 0.6843 |
| Linear SVC          |           | 0.1     | 0.8105 | 0.6642 |
| Logistic Regression | lbfgs     | 1       | 0.7517 | 0.6322 |
| Logistic Regression | newton-cg | 1       | 0.7517 | 0.6322 |
| Linear SVC          |           | 0.01    | 0.5205 | 0.4569 |
| Logistic Regression | lbfgs     | 0.1     | 0.4171 | 0.3786 |
| Logistic Regression | lbfgs     | 0.001   | 0.1384 | 0.1355 |

After tuning, the best Linear SVC model is generating an accuracy percentage of 74.19% on the test set and the best Logistic Regression Classifier is generating an accuracy percentage of 72.95%.  Both models are performing well at this point, especially considering that there are approximately 150 judges who authored opinions in the dataset.

As a final step, I will add some `SpaCy` word embeddings vectors for each opinion to see if they improve model performance.

### SpaCy Vectors

The `SpaCy` package converts documents into 300 dimension vectors that attempt to capture the substance of the text with numerical values.  I have previously created these vectors and saved them to a local file, so I will import that file and then add the vectors as additional features to the `df_ml` dataframe.  I have provided to code I used to create the vectors in the next cell (as comments because I do not need to run the code at this time).

In [ ]:
# import and instantiate SpaCy
# import spacy
# nlp = spacy.load('en_core_web_md')

# create spacy vectors
# from tqdm import tqdm
# for row in tqdm(range(len(df_spacy))):
    #doc = nlp(df.loc[row, 'opinion'])
    #for i in range(300):
        #df_spacy.loc[row, 'spacy_{}'.format(i)] = doc.vector[i]

In [4]:
# import file
df_spacy = pd.read_csv('df_spacy.csv', index_col=0)

In [7]:
df_spacy.head()

,spacy_0,spacy_1,spacy_2,spacy_3,spacy_4,spacy_5,spacy_6,spacy_7,spacy_8,spacy_9,spacy_10,spacy_11,spacy_12,spacy_13,spacy_14,spacy_15,spacy_16,spacy_17,spacy_18,spacy_19,spacy_20,spacy_21,spacy_22,spacy_23,spacy_24,spacy_25,spacy_26,spacy_27,spacy_28,spacy_29,spacy_30,spacy_31,spacy_32,spacy_33,spacy_34,spacy_35,spacy_36,spacy_37,spacy_38,spacy_39,spacy_40,spacy_41,spacy_42,spacy_43,spacy_44,spacy_45,spacy_46,spacy_47,spacy_48,spacy_49,spacy_50,spacy_51,spacy_52,spacy_53,spacy_54,spacy_55,spacy_56,spacy_57,spacy_58,spacy_59,spacy_60,spacy_61,spacy_62,spacy_63,spacy_64,spacy_65,spacy_66,spacy_67,spacy_68,spacy_69,spacy_70,spacy_71,spacy_72,spacy_73,spacy_74,spacy_75,spacy_76,spacy_77,spacy_78,spacy_79,spacy_80,spacy_81,spacy_82,spacy_83,spacy_84,spacy_85,spacy_86,spacy_87,spacy_88,spacy_89,spacy_90,spacy_91,spacy_92,spacy_93,spacy_94,spacy_95,spacy_96,spacy_97,spacy_98,spacy_99,spacy_100,spacy_101,spacy_102,spacy_103,spacy_104,spacy_105,spacy_106,spacy_107,spacy_108,spacy_109,spacy_110,spacy_111,spacy_112,spacy_113,spacy_114,spacy_115,spacy_116,spacy_117,spacy_118,spacy_119,spacy_120,spacy_121,spacy_122,spacy_123,spacy_124,spacy_125,spacy_126,spacy_127,spacy_128,spacy_129,spacy_130,spacy_131,spacy_132,spacy_133,spacy_134,spacy_135,spacy_136,spacy_137,spacy_138,spacy_139,spacy_140,spacy_141,spacy_142,spacy_143,spacy_144,spacy_145,spacy_146,spacy_147,spacy_148,spacy_149,spacy_150,spacy_151,spacy_152,spacy_153,spacy_154,spacy_155,spacy_156,spacy_157,spacy_158,spacy_159,spacy_160,spacy_161,spacy_162,spacy_163,spacy_164,spacy_165,spacy_166,spacy_167,spacy_168,spacy_169,spacy_170,spacy_171,spacy_172,spacy_173,spacy_174,spacy_175,spacy_176,spacy_177,spacy_178,spacy_179,spacy_180,spacy_181,spacy_182,spacy_183,spacy_184,spacy_185,spacy_186,spacy_187,spacy_188,spacy_189,spacy_190,spacy_191,spacy_192,spacy_193,spacy_194,spacy_195,spacy_196,spacy_197,spacy_198,spacy_199,spacy_200,spacy_201,spacy_202,spacy_203,spacy_204,spacy_205,spacy_206,spacy_207,spacy_208,spacy_209,spacy_210,spacy_211,spacy_212,spacy_213,spacy_214,spacy_215,spacy_216,spacy_217,spacy_218,spacy_219,spacy_220,spacy_221,spacy_222,spacy_223,spacy_224,spacy_225,spacy_226,spacy_227,spacy_228,spacy_229,spacy_230,spacy_231,spacy_232,spacy_233,spacy_234,spacy_235,spacy_236,spacy_237,spacy_238,spacy_239,spacy_240,spacy_241,spacy_242,spacy_243,spacy_244,spacy_245,spacy_246,spacy_247,spacy_248,spacy_249,spacy_250,spacy_251,spacy_252,spacy_253,spacy_254,spacy_255,spacy_256,spacy_257,spacy_258,spacy_259,spacy_260,spacy_261,spacy_262,spacy_263,spacy_264,spacy_265,spacy_266,spacy_267,spacy_268,spacy_269,spacy_270,spacy_271,spacy_272,spacy_273,spacy_274,spacy_275,spacy_276,spacy_277,spacy_278,spacy_279,spacy_280,spacy_281,spacy_282,spacy_283,spacy_284,spacy_285,spacy_286,spacy_287,spacy_288,spacy_289,spacy_290,spacy_291,spacy_292,spacy_293,spacy_294,spacy_295,spacy_296,spacy_297,spacy_298,spacy_299
0,-0.018235,0.141126,-0.077766,-0.041140,-0.001164,-0.014620,0.032196,-0.082413,0.024770,2.116468,-0.192804,0.049729,0.141365,-0.016655,-0.169243,-0.062276,-0.033327,1.070306,-0.115193,-0.033005,0.031212,-0.071661,-0.031958,-0.008337,0.065750,0.043408,-0.052442,-0.077462,-0.011728,-0.059205,-0.040694,0.066518,-0.024291,0.079856,0.069881,-0.082784,-0.011462,-0.000150,-0.071227,-0.070347,-0.024128,0.029422,0.077631,-0.088816,-0.031311,-0.014032,-0.116316,0.063228,-0.001043,0.004374,-0.039147,0.051738,-0.047044,0.039755,-0.034598,0.012541,0.026246,-0.030908,0.002282,-0.062070,-0.011083,-0.026285,0.018029,0.182286,0.006683,-0.005607,0.001397,0.007796,-0.047747,0.035894,0.072234,0.042888,0.101939,-0.074478,0.132518,-0.080515,0.009667,-0.016727,-0.011491,0.109113,0.003445,0.073345,-0.115607,0.012601,0.034442,-0.132139,-0.031270,-0.150163,0.177718,0.060418,-0.068316,-0.028899,0.010970,-0.024210,0.116330,-0.069125,-0.071684,-0.036820,0.014122,0.003569,0.067058,0.043579,-0.044930,0.038197,0.051132,-0.695979,0.049199,0.030121,0.033530,0.001616,0.020328,0.041632,-0.005094,-0.095717,-0.009560,0.008061,0.

In [6]:
del df_spacy['id']
del df_spacy['judge']

In [8]:
df_ml.head()

,id,judge,type_concurrence,type_concurring-in-part-and-dissenting-in-part,type_dissent,type_majority,type_rehearing,year_1779,year_1784,year_1787,year_1789,year_1790,year_1791,year_1792,year_1793,year_1794,year_1795,year_1796,year_1797,year_1798,year_1799,year_1800,year_1801,year_1802,year_1803,year_1804,year_1805,year_1806,year_1807,year_1808,year_1809,year_1810,year_1811,year_1812,year_1813,year_1814,year_1815,year_1816,year_1817,year_1818,year_1819,year_1820,year_1821,year_1822,year_1823,year_1824,year_1825,year_1826,year_1827,year_1828,year_1829,year_1830,year_1831,year_1832,year_1833,year_1834,year_1835,year_1836,year_1837,year_1838,year_1839,year_1840,year_1841,year_1842,year_1843,year_1844,year_1845,year_1846,year_1847,year_1848,year_1849,year_1850,year_1851,year_1852,year_1853,year_1854,year_1855,year_1856,year_1857,year_1858,year_1859,year_1860,year_1861,year_1862,year_1863,year_1864,year_1866,year_1867,year_1868,year_1869,year_1870,year_1871,year_1872,year_1873,year_1874,year_1875,year_1876,year_1877,year_1878,year_1879,year_1880,year_1881,year_1882,year_1883,year_1884,year_1885,year_1886,year_1887,year_1888,year_1889,year_1890,year_1891,year_1892,year_1893,year_1894,year_1895,year_1896,year_1897,year_1898,year_1899,year_1900,year_1901,year_1902,year_1903,year_1904,year_1905,year_1906,year_1907,year_1908,year_1909,year_1910,year_1911,year_1912,year_1913,year_1914,year_1915,year_1916,year_1917,year_1918,year_1919,year_1920,year_1921,year_1922,year_1923,year_1924,year_1925,year_1926,year_1927,year_1928,year_1929,year_1930,year_1931,year_1932,year_1933,year_1934,year_1935,year_1936,year_1937,year_1938,year_1939,year_1940,year_1941,year_1942,year_1943,year_1944,year_1945,year_1946,year_1947,year_1948,year_1949,year_1950,year_1951,year_1952,year_1953,year_1954,year_1955,year_1956,year_1957,year_1958,year_1959,year_1960,year_1961,year_1962,year_1963,year_1964,year_1965,year_1966,year_1967,year_1968,year_1969,year_1970,year_1971,year_1972,year_1973,year_1974,year_1975,year_1976,year_1977,year_1978,year_1979,year_1980,year_1981,year_1982,year_1983,year_1984,year_1985,year_1986,year_1987,year_1988,year_1989,year_1990,year_1991,year_1992,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,word_count,sentence_count,avg_sent_length,polarity,subjectivity
0,53839,frye,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.088804,0.098247,0.093527,0.501317,0.473721
1,53841,per_curiam,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000265,0.000923,0.018977,0.444444,0.000000
2,53843,parker_sarah,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [9]:
# add the spacy word embedded vectors to the df_ml dataframe
df_ml = pd.concat([df_ml, df_tfidf, df_spacy], axis=1)
df_ml.head()

,id,judge,type_concurrence,type_concurring-in-part-and-dissenting-in-part,type_dissent,type_majority,type_rehearing,year_1779,year_1784,year_1787,year_1789,year_1790,year_1791,year_1792,year_1793,year_1794,year_1795,year_1796,year_1797,year_1798,year_1799,year_1800,year_1801,year_1802,year_1803,year_1804,year_1805,year_1806,year_1807,year_1808,year_1809,year_1810,year_1811,year_1812,year_1813,year_1814,year_1815,year_1816,year_1817,year_1818,year_1819,year_1820,year_1821,year_1822,year_1823,year_1824,year_1825,year_1826,year_1827,year_1828,year_1829,year_1830,year_1831,year_1832,year_1833,year_1834,year_1835,year_1836,year_1837,year_1838,year_1839,year_1840,year_1841,year_1842,year_1843,year_1844,year_1845,year_1846,year_1847,year_1848,year_1849,year_1850,year_1851,year_1852,year_1853,year_1854,year_1855,year_1856,year_1857,year_1858,year_1859,year_1860,year_1861,year_1862,year_1863,year_1864,year_1866,year_1867,year_1868,year_1869,year_1870,year_1871,year_1872,year_1873,year_1874,year_1875,year_1876,year_1877,year_1878,year_1879,year_1880,year_1881,year_1882,year_1883,year_1884,year_1885,year_1886,year_1887,year_1888,year_1889,year_1890,year_1891,year_1892,year_1893,year_1894,year_1895,year_1896,year_1897,year_1898,year_1899,year_1900,year_1901,year_1902,year_1903,year_1904,year_1905,year_1906,year_1907,year_1908,year_1909,year_1910,year_1911,year_1912,year_1913,year_1914,year_1915,year_1916,year_1917,year_1918,year_1919,year_1920,year_1921,year_1922,year_1923,year_1924,year_1925,year_1926,year_1927,year_1928,year_1929,year_1930,year_1931,year_1932,year_1933,year_1934,year_1935,year_1936,year_1937,year_1938,year_1939,year_1940,year_1941,year_1942,year_1943,year_1944,year_1945,year_1946,year_1947,year_1948,year_1949,year_1950,year_1951,year_1952,year_1953,year_1954,year_1955,year_1956,year_1957,year_1958,year_1959,year_1960,year_1961,year_1962,year_1963,year_1964,year_1965,year_1966,year_1967,year_1968,year_1969,year_1970,year_1971,year_1972,year_1973,year_1974,year_1975,year_1976,year_1977,year_1978,year_1979,year_1980,year_1981,year_1982,year_1983,year_1984,year_1985,year_1986,year_1987,year_1988,year_1989,year_1990,year_1991,year_1992,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,word_count,sentence_count,avg_sent_length,polarity,subjectivity,00,000,10,100,101,102,103,...,spacy_50,spacy_51,spacy_52,spacy_53,spacy_54,spacy_55,spacy_56,spacy_57,spacy_58,spacy_59,spacy_60,spacy_61,spacy_62,spacy_63,spacy_64,spacy_65,spacy_66,spacy_67,spacy_68,spacy_69,spacy_70,spacy_71,spacy_72,spacy_73,spacy_74,spacy_75,spacy_76,spacy_77,spacy_78,spacy_79,spacy_80,spacy_81,spacy_82,spacy_83,spacy_84,spacy_85,spacy_86,spacy_87,spacy_88,spacy_89,spacy_90,spacy_91,spacy_92,spacy_93,spacy_94,spacy_95,spacy_96,spacy_97,spacy_98,spacy_99,spacy_100,spacy_101,spacy_102,spacy_103,spacy_104,spacy_105,spacy_106,spacy_107,spacy_108,spacy_109,spacy_110,spacy_111,spacy_112,spacy_113,spacy_114,spacy_115,spacy_116,spacy_117,spacy_118,spacy_119,spacy_120,spacy_121,spacy_122,spacy_123,spacy_124,spacy_125,spacy_126,spacy_127,spacy_128,spacy_129,spacy_130,spacy_131,spacy_132,spacy_133,spacy_134,spacy_135,spacy_136,spacy_137,spacy_138,spacy_139,spacy_140,spacy_141,spacy_142,spacy_143,spacy_144,spacy_145,spacy_146,spacy_147,spacy_148,spacy_149,spacy_150,spacy_151,spacy_152,spacy_153,spacy_154,spacy_155,spacy_156,spacy_157,spacy_158,spacy_159,spacy_160,spacy_161,spacy_162,spacy_163,spacy_164,spacy_165,spacy_166,spacy_167,spacy_168,spacy_169,spacy_170,spacy_171,spacy_172,spacy_173,spacy_174,spacy_175,spacy_176,spacy_177,spacy_178,spacy_179,spacy_180,spacy_181,spacy_182,spacy_183,spacy_184,spacy_185,spacy_186,spacy_187,spacy_188,spacy_189,spacy_190,spacy_191,spacy_192,spacy_193,spacy_194,spacy_195,spacy_196,spacy_197,spacy_198,spacy_199,spacy_200,spacy_201,spacy_202,

In [10]:
# divide the data into label and features for use in ml models
X = df_ml.iloc[:, 2:]
y = df_ml.loc[:, 'judge']

# split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, stratify=y)

### Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=100, solver='lbfgs', multi_class='auto', max_iter=3000, random_state=6)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print('Accuracy on training set = {}'.format(lr.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(lr.score(X_test, y_test)))

C:\Users\jacob\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy on training set = 0.9999339508924886
Accuracy on test set = 0.7542599564097484


### Linear Support Vector Machine Classifier

In [12]:
from sklearn.svm import LinearSVC
svc = LinearSVC(max_iter=1500, random_state=6)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print('Accuracy on training set = {}'.format(svc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(svc.score(X_test, y_test)))

Accuracy on training set = 0.9746866795462427
Accuracy on test set = 0.7675847037844263


The `SpaCy` vectors improved the accuracy of both models by approximately 2%, demonstrating that the substance of the opinions provides signal for the models.  Here are the final results:

| Model               | Train  | Test   |
|---------------------|--------|--------|
| Linear SVC          | 0.9747 | 0.7676 |
| Logistic Regression | 0.9999 | 0.7542 |

With both models generating an accuracy percentage above 75%, I have succeeded in building two models that will provide value to my clients.